In [13]:
# decorator class is a class that can be used as a decorator

from functools import wraps

def logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        print(f"log {fn.__name__} called...")
        return fn(*args, **kwargs)
    return inner


@logger
def say_hello():
    pass

In [14]:
say_hello()

log say_hello called...


In [15]:
help(say_hello)

Help on function say_hello in module __main__:

say_hello()



In [22]:
class Logger:
    """
    Class that can be used as a decorator.
    Using it as such will cause decorated function to be an instance
    of a decorator. That means that this instance should be in line 
    with decorators interface - should be callable and implement
    non data descriptor interface __get__ method.
    """
    def __init__(self, fn):
        self.fn = fn

    def __call__(self, *args, **kwargs):
        print(f"__call__ called for {self.fn}")
        return self.fn(*args, **kwargs)

    # commented out for now
    # def __get__(self, *args, **kwargs):
    #     pass


In [23]:
def say_hello():
    pass

f = Logger(say_hello)  # f is an instance of the Logger class
f()

__call__ called for <function say_hello at 0x105cf2020>


In [24]:
@Logger
def say_hello():
    return "hello"

say_hello()

__call__ called for <function say_hello at 0x105cf1e40>


'hello'

In [25]:
type(say_hello), isinstance(say_hello, Logger)

(__main__.Logger, True)

In [26]:
class Person:
    def __init__(self, name):
        self.name = name

    @Logger
    def say_hello(self):
        return f"{self.name} says hi!"


In [31]:
p = Person("Bob")
print(type(p.say_hello))  # it's not a method now! so self won't be passeed
try:
    p.say_hello()
except TypeError as e:
    print(e)

<class '__main__.Logger'>
__call__ called for <function Person.say_hello at 0x105cf2340>
Person.say_hello() missing 1 required positional argument: 'self'


In [50]:
from types import MethodType


class Logger:
    """
        We need to implement __get__ method ourselfs.
        In addition, we need to bound decorated function to an instance,
        so we need to create a method!
    """
    def __init__(self, fn):
        self.fn = fn

    def __call__(self, *args, **kwargs):
        print(f"Logger: __call__ called for {self.fn}")
        return self.fn(*args, **kwargs)

    def __get__(self, instance, owner_class):
        # implements non data descriptor
        print(f"__get__ called self={self}, instance={instance}")
        if instance is None:
            print("rerturning self unbound")
            return self
        print(f"returning self as a method bound to an instance")
        # bounds `self` (instance of a Logger that is callable) 
        # to the class instance of a method that is being decorated
        return MethodType(self, instance)


In [51]:
class Person:
    def __init__(self, name):
        self.name = name

    @Logger
    def say_hello(self):
        return f"{self.name} says hello!"


In [52]:
p = Person("Bob")
p.__dict__

{'name': 'Bob'}

In [53]:
Person.say_hello

__get__ called self=<__main__.Logger object at 0x105867b90>, instance=None
rerturning self unbound


In [54]:
p.say_hello

__get__ called self=<__main__.Logger object at 0x105867b90>, instance=<__main__.Person object at 0x1058641d0>
returning self as a method bound to an instance


<bound method ? of <__main__.Person object at 0x1058641d0>>

In [55]:
p.say_hello()

__get__ called self=<__main__.Logger object at 0x105867b90>, instance=<__main__.Person object at 0x1058641d0>
returning self as a method bound to an instance
Logger: __call__ called for <function Person.say_hello at 0x105cf3380>


'Bob says hello!'

In [56]:
@Logger
def say_bye():
    pass


In [57]:
type(say_bye)

__main__.Logger

In [59]:
say_bye()  # also works, because Python isn't looking for a descriptor

Logger: __call__ called for <function say_bye at 0x105cf13a0>


In [62]:
class Person:
    def __init__(self, name):
        self._name = name

    @classmethod
    @Logger
    def cls_method(cls):
        return "Hi from a classmethod!"

    @staticmethod
    @Logger
    def static_method():
        return "Hi for a staticmethod!"


    @property
    @Logger
    def name(self):
        return self._name

In [64]:
p = Person("Bob")
p.cls_method()

__get__ called self=<__main__.Logger object at 0x105864fe0>, instance=<class '__main__.Person'>
returning self as a method bound to an instance
Logger: __call__ called for <function Person.cls_method at 0x105cf0860>


'Hi from a classmethod!'

In [65]:
p.static_method()

Logger: __call__ called for <function Person.static_method at 0x105cf3600>


'Hi for a staticmethod!'

In [66]:
p.name

Logger: __call__ called for <function Person.name at 0x105cf0d60>


'Bob'